# Setting up environment

Install packages

In [1]:
%pip install --upgrade datasets transformers ninja decord timm accelerate
%pip install flash-attn --no-build-isolation


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Using InternVL3 one time

Import packages

In [2]:
import time
import csv
import logging
import math
import torch
import torchvision.transforms as T
import numpy as np
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForCausalLM
from huggingface_hub import snapshot_download

Split model

In [3]:
def split_model(model_path):
    device_map = {}
    world_size = torch.cuda.device_count()
    config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers
    # Since the first GPU will be used for ViT, treat it as half a GPU.
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.model.rotary_emb'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

Load dataset and model

In [4]:
dataset = load_dataset("lmms-lab/AISG_Challenge")
snapshot_download("OpenGVLab/InternVL3-78B", local_dir="./InternVL3-78B")
model = AutoModel.from_pretrained(
    "./InternVL3-78B",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
    device_map=split_model('./InternVL3-78B'),
    local_files_only=True).eval()
tokenizer = AutoTokenizer.from_pretrained('./InternVL3-78B', trust_remote_code=True, use_fast=False, local_files_only=True)
text_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-7B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="cuda")
text_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
generation_config = dict(max_new_tokens=1024, do_sample=False)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

Fetching 53 files:   0%|          | 0/53 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Download video function

In [5]:
def retrieve_video(video_id):
    filename = f"{video_id}.mp4"
    video_path = f"./videos/videos/{filename}"
    return video_path

Improve prompt

In [6]:
def improve_question(question):
    if len(question.split('\n')) <= 1:
        actual_question = question.split('\n')[0]
        
        prompt = f"Generalise the question using only one 5W1H method (Who, What, When, Where, Why, and How). Focus on the main idea of the question instead of specific assumptions. Only return the rewritten question.\nOriginal question: {actual_question}\nRewritten question:"
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = text_tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = text_tokenizer([text], return_tensors="pt").to(text_model.device)
        generated_ids = text_model.generate(
            **model_inputs,
            max_new_tokens=128,
            do_sample=False,
            temperatue=None,
            top_p=None,
            top_k=None
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = text_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        return response + " " + question
    else:
        return question

Narrow down question scope

In [7]:
def narrow(question):
    actual_question = question.split('\n')[0]
    
    prompt = f"Break down the question into object, subject, temporal or spatial information in the following format:\nobject, subject, temporal, spatial\nThe question is: {actual_question}."
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = text_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = text_tokenizer([text], return_tensors="pt").to(text_model.device)
    generated_ids = text_model.generate(
        **model_inputs,
        max_new_tokens=256,
        do_sample=False,
        temperatue=None,
        top_p=None,
        top_k=None
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = text_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

Build transform

In [8]:
def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

Find closest aspect ratio

In [9]:
def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

Dynamic preprocess

In [10]:
def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

Get index

In [11]:
def get_index(bound, fps, max_frame, first_idx=0, num_segments=32):
    if bound:
        start, end = bound[0], bound[1]
    else:
        start, end = -100000, 100000
    start_idx = max(first_idx, round(start * fps))
    end_idx = min(round(end * fps), max_frame)
    seg_size = float(end_idx - start_idx) / num_segments
    frame_indices = np.array([
        int(start_idx + (seg_size / 2) + np.round(seg_size * idx))
        for idx in range(num_segments)
    ])
    return frame_indices

Load video

In [12]:
def load_video(video_path, bound=None, input_size=448, max_num=1, num_segments=32):
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    max_frame = len(vr) - 1
    fps = float(vr.get_avg_fps())

    pixel_values_list, num_patches_list = [], []
    transform = build_transform(input_size=input_size)
    frame_indices = get_index(bound, fps, max_frame, first_idx=0, num_segments=num_segments)
    for frame_index in frame_indices:
        img = Image.fromarray(vr[frame_index].asnumpy()).convert('RGB')
        img = dynamic_preprocess(img, image_size=input_size, use_thumbnail=True, max_num=max_num)
        pixel_values = [transform(tile) for tile in img]
        pixel_values = torch.stack(pixel_values)
        num_patches_list.append(pixel_values.shape[0])
        pixel_values_list.append(pixel_values)
    pixel_values = torch.cat(pixel_values_list)
    return pixel_values, num_patches_list

Process sample

In [13]:
def process_test_case(example):
    video_id = example["video_id"]
    question = example["question"]
    question_prompt = example["question_prompt"]
    
    video_path = retrieve_video(video_id)
    improved_question = improve_question(question)

    pixel_values, num_patches_list = load_video(video_path, num_segments=32, max_num=1)
    pixel_values = pixel_values.to(torch.bfloat16).cuda()
    video_prefix = ''.join([f'Frame{i+1}: <image>\n' for i in range(len(num_patches_list))])

    prompt = video_prefix + f"\nFind the following information from the video:\n{narrow(question)}"
    response, history = model.chat(tokenizer, pixel_values, prompt, generation_config, num_patches_list=num_patches_list, history=None, return_history=True)
    '''
    print(f"1st Prompt:\n{prompt}")
    print(f"1st Response:\n{response}")
    '''
    prompt = video_prefix + f"\n{improved_question}\n{question_prompt}"
    response, history = model.chat(tokenizer, pixel_values, prompt, generation_config, num_patches_list=num_patches_list, history=history, return_history=True)
    '''
    print(f"2nd Prompt:\n{prompt}")
    print(f"2nd Response:\n{response}")
    print(f"Video URL: {example['youtube_url']}")
    '''
    return example['qid'], response

Run test cases

In [ ]:
start_time = time.time()
output_file = "./results.csv"
start_processing = False
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["qid", "pred"])
    '''
    sample = dataset['test'].filter(lambda x: x['qid'] == '0340-0')[0]
    qid, pred = process_test_case(sample)
    writer.writerow([qid, pred])
    '''
    for sample in dataset['test']:
        if not start_processing:
            if sample['qid'] == "0008-0":
                start_processing = True
            else:
                continue
        logging.debug(f"processing qid {sample['qid']}")
        qid, pred = process_test_case(sample)
        writer.writerow([qid, pred])
    
    end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

DEBUG:root:processing qid 0008-0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (8548 > 8192). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
DEBUG:root:processing qid 0008-1
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
DEBUG:root:processing qid 0008-2
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151